In [1]:
import tensorflow as tf
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np



# the entire dataset is loaded
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


To begin with, we import the various libraries including the CIFAR-10 dataset that will be used in the partitioning

In [2]:
x_data = np.concatenate((x_train, x_test), axis=0)
y_data = np.concatenate((y_train, y_test), axis=0)

In [3]:
num_samples = x_data.shape[0]

Since the CIFAR-10 already has 50,000 training and 10,000 testing images. We need to split these 2 sets to create a validation set.
The code above concatenates(adds) the train and test into the x_data which will be used in the split

In [4]:
train_size = int(0.70 * num_samples)  # 70% for training
val_size = int(0.15 * num_samples)  # 15% for validation
test_size = num_samples - train_size - val_size  # 15% for testing


In [ ]:
In the above we define the split ratio which will be 70/15/15

SyntaxError: invalid syntax (<ipython-input-6-f60168c1c663>, line 1)

In [5]:
x_train = x_data[:train_size]
y_train = y_data[:train_size]
x_val = x_data[train_size:train_size+val_size]
y_val = y_data[train_size:train_size+val_size]
x_test = x_data[train_size+val_size:]
y_test = y_data[train_size+val_size:]

In [ ]:
THe above code, then splits the x_data and y_data and placing them into the respective train, validation and test sets.

In [6]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
print(f"Train Data: {len(x_train)}")
print(f"Validation Data: {len(x_val)}")
print(f"Test Data: {len(x_test)}")

Train Data: 42000
Validation Data: 9000
Test Data: 9000


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [13]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.25)

model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.25)


model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5)
model.add(Dense(10, activation='softmax'))




In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In the above code, the categorical cross entropy loss function will be used.

In [15]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 15, 15, 64)        36928     
                                                                 
 conv2d_9 (Conv2D)           (None, 13, 13, 64)        36928     
                                                      

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test), shuffle=True)

#Saving the neural network
model_structure = model.to_json()
with open("model_structure.json", "w") as json_file:
    json_file.write(model_structure)

#Save the trained weights
model.save_weights("model_weights.h5")

Epoch 1/30
1313/1313 [==============================] - 295s 224ms/step - loss: 1.6504 - accuracy: 0.4384 - val_loss: 1.2968 - val_accuracy: 0.5374
Epoch 2/30
1313/1313 [==============================] - 311s 237ms/step - loss: 1.1504 - accuracy: 0.5913 - val_loss: 1.1268 - val_accuracy: 0.6076
Epoch 3/30
1313/1313 [==============================] - 318s 242ms/step - loss: 0.9277 - accuracy: 0.6732 - val_loss: 1.0889 - val_accuracy: 0.6293
Epoch 4/30
1313/1313 [==============================] - 314s 239ms/step - loss: 0.7280 - accuracy: 0.7446 - val_loss: 1.2079 - val_accuracy: 0.6148
Epoch 5/30
1313/1313 [==============================] - 333s 254ms/step - loss: 0.5128 - accuracy: 0.8200 - val_loss: 1.3292 - val_accuracy: 0.6307
Epoch 6/30
1313/1313 [==============================] - 326s 248ms/step - loss: 0.3505 - accuracy: 0.8809 - val_loss: 1.8367 - val_accuracy: 0.6031
Epoch 7/30
1313/1313 [==============================] - 316s 241ms/step - loss: 0.2795 - accuracy: 0.9076 - val_